In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

# Orientations

In [ ]:
import numpy as np
from materialite import Orientation, Material

`Orientation`s in Materialite represent a basis transformation from one coordinate frame to another. In the language of materials science, the transformation is from a "specimen" (or "sample" or "lab") frame to a "crystal" frame.

Make an `Orientation` representing the Goss orientation. We'll use the `from_euler_angles` method, which expects a set of Bunge Euler angles in radians. The Bunge Euler angles represent a rotation about the specimen `z` axis, followed by a rotation about the new `x` axis, followed by a rotation about the new `z` axis. The three angles are often defined as $\phi_1$, $\Phi$, and $\phi_2$, respectively.

As an example, create the Goss orientation, which requires only a $\pi/4$ or $45^\circ$ rotation about the `x` axis.

In [ ]:
orientation = Orientation.from_euler_angles([0.0, np.pi / 4, 0.0])
print(f"rotation matrix:\n {orientation.rotation_matrix}")
print(f"Bunge Euler angles: {orientation.euler_angles}")
print(f"Bunge Euler angles in degrees: {orientation.euler_angles_in_degrees}")

The rows of the rotation matrix represent the crystal basis vectors expressed in the specimen frame. The columns represent the specimen basis vectors expressed in the crystal frame. Here, the specimen $[100]$ direction is aligned with the crystal $[100]$ direction, the specimen $[010]$ direction is aligned with the crystal $[01\bar{1}]$ direction, and the specimen $[001]$ direction is aligned with the crystal $[011]$ direction.

We can also construct the `Orientation` using the Euler angles in degrees if we set the `in_degrees` argument to `True`.

In [ ]:
orientation = Orientation.from_euler_angles([0.0, 45.0, 0.0], in_degrees=True)
print(f"Bunge Euler angles in degrees: {orientation.euler_angles_in_degrees}")

`Orientation`s can have dimensions, just like `Tensor`s (see the `Tensor` demo for details). The default representation of an `Orientation` is the Euler angles in radians.

In [ ]:
orientations = Orientation.from_euler_angles(
    [[0.0, 45.0, 0.0], [35.0, 45.0, 0.0]], in_degrees=True
)
orientations

We can also generate random orientations.

In [ ]:
rng = np.random.default_rng(0)
orientations = Orientation.random(shape=1, rng=rng)
print(orientations)
orientations = Orientation.random(shape=3, rng=rng)
print(orientations)

We can pass a multi-dimensional shape to `Orientation.random` as well, which will produce an `Orientation` with an additional dimension. Like with `Tensor`s, the first two dimensions are "points" and "slip systems" by default, but this can be changed by specifying a string for the `dims` argument.

In [ ]:
rng = np.random.default_rng(0)
orientations = Orientation.random(shape=(3, 2), rng=rng)
print(orientations)
print("\n")
orientations = Orientation.random(shape=(3, 2), rng=rng, dims="pa")
print(orientations)

Note that the Euler angles are always mapped to $\ -\pi \le \phi_1 < \pi,\ 0 \le \Phi < \pi,\ $ and $\ -\pi \le \phi_2 < \pi$.

We can add `Orientation`s as fields to a `Material` in several ways. Assign the identity `Orientation` to all points (useful for simple constitutive models):

In [ ]:
Material(dimensions=[2, 2, 2]).create_uniform_field(
    "orientation", Orientation.identity()
).get_fields()

Assign the same random `Orientation` to all points:

In [ ]:
Material(dimensions=[2, 2, 2]).create_uniform_field(
    "orientation", Orientation.random(1)
).get_fields()

Assign a different random `Orientation` to each point:

In [ ]:
orientations = Orientation.random(8)
Material(dimensions=[2, 2, 2]).create_fields({"orientation": orientations}).get_fields()

`Orientation`s can be composed using the `@` operator. Just like `Tensor`s, the `@` operator will work with arbitrary dimensions.

In [ ]:
orientation1 = Orientation.from_euler_angles([0.0, 60.0, 0.0], in_degrees=True)
orientation2 = Orientation.from_euler_angles([0.0, 30.0, 0.0], in_degrees=True)
print((orientation1 @ orientation2))
orientation3 = Orientation.from_euler_angles(
    [[0.0, 30.0, 0.0], [45.0, 0.0, 0.0]], in_degrees=True
)
print((orientation1 @ orientation3))

`Orientation`s can also be constructed from Miller indices. Here, we provide the crystal plane normal that is parallel to the specimen ND/[001]/z direction and the crystal direction that is parallel to the RD/[100]/x direction. As an example, we will make the Goss orientation again.

In [ ]:
Orientation.from_miller_indices(
    plane=[0, 1, 1], direction=[1, 0, 0]
).euler_angles_in_degrees

You can also provide multiple pairs of planes and directions. Here, we'll create two variants of the Goss orientation.

In [ ]:
Orientation.from_miller_indices(
    plane=[[0, 1, 1], [0, 1, 1]], direction=[[1, 0, 0], [0, 0, 1]]
).euler_angles_in_degrees